In [1]:
# Standard libraries
import h5py
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive

# Append base directory
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
path1p = os.path.dirname(currentdir)
libpath = os.path.join(path1p, "lib")

sys.path.insert(0, libpath) 
print("Appended library directory", libpath)

# User libraries
from models.test_lib import noisePure, noiseLPF, sampleTrials
from models.dyn_sys import DynSys

from fc.te_idtxl_wrapper import idtxlParallelCPU, idtxlResultsParse

%load_ext autoreload
%autoreload 2

/opt/anaconda3/envs/py36qt5/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Appended library directory /media/aleksejs/DataHDD/work/codes/comp-neuro/analysis-mesoscopic/mesoscopic-functional-connectivity/codes/lib


In [2]:
def matdiff(m1, m2):
    TP = FP = FN = TN = 0
    for i in range(m1.shape[0]):
        for j in range(m1.shape[1]):
            NAN1 = np.isnan(m1[i, j])
            NAN2 = np.isnan(m2[i, j])
            TP += (not NAN1) and (not NAN2)
            FP += (not NAN1) and (NAN2)
            FN += (NAN1) and (not NAN2)
            TN += (NAN1) and (NAN2)
            
    T1 = FP / (FP + TP) if FP + TP != 0 else 0
    T2 = FN / (FN + TN) if FN + TN != 0 else 0
    FP_RATE = FP / np.prod(m1.shape)
    FN_RATE = FN / np.prod(m1.shape)
            
    return TP,FP,FN,TN,T1,T2,FP_RATE,FN_RATE

def testplots(ndata, te_data_combo, TRUE_CONN, logx=True, pfilter=None, h5_fname=None, fig_fname=None):
    te_3D, lag_3D, p_3D = te_data_combo
    
    #####################################
    # Save data
    #####################################
    if h5_fname is not None:
        with h5py.File(h5_fname, "w") as h5f:
            grp_rez = h5f.create_group("results")
            grp_rez['TE_table'] = te_3D
            grp_rez['delay_table'] = lag_3D
            grp_rez['p_table'] = p_3D
    
    #####################################
    # Analysis
    #####################################
    
    # Copy data to avoid modifying originals
    te, lag, p = np.copy(te_3D), np.copy(lag_3D), np.copy(p_3D)
    NSTEP = te.shape[2]
    
    # Delete all connections that have too high p-value, if requested
    if pfilter is not None:
        # Find which non-nan entries of p-value need to be filtered
        mask = ~np.isnan(p)
        mask[mask] &= p[mask] > pfilter
        
        # Set those entries to nan
        te[mask] = np.nan
        lag[mask] = np.nan
        p[mask] = np.nan
    
    # Compute statistics
    test_fp = np.array([matdiff(te[:, :, i], TRUE_CONN) for i in range(NSTEP)]).transpose()
    freq_conn = NSTEP - np.sum(np.isnan(te), axis=2)
    
    #####################################
    # Plots
    #####################################
    fig, ax = plt.subplots(nrows = 2, ncols = 3, figsize=(15,10))
    ax[0][0].set_title("Error rates")
    if logx:
        #ax[0][0].semilogx(ndata, test_fp[4], label='T1')
        #ax[0][0].semilogx(ndata, test_fp[5], label='T2')
        ax[0][0].semilogx(ndata, test_fp[6], '.-', label='FP_RATE')
        ax[0][0].semilogx(ndata, test_fp[7], '.-', label='FN_RATE')
    else:
        #ax[0][0].plot(ndata, test_fp[4], label='T1')
        #ax[0][0].plot(ndata, test_fp[5], label='T2')
        ax[0][0].plot(ndata, test_fp[6], '.-', label='FP_RATE')
        ax[0][0].plot(ndata, test_fp[7], '.-', label='FN_RATE')
    ax[0][0].legend()
    
    ax[0][1].set_title("True connections")
    ax[0][1].imshow(TRUE_CONN)
    
    ax[1][1].set_title("Frequencies of connections")
    ax[1][1].imshow(freq_conn, vmin=0, vmax=NSTEP)
    
    ax[1][0].set_title("TE for each connection")
    ax[1][2].set_title("p for each connection")
    ax[0][2].set_title("lag for each connection")
    ax[0][2].set_yticks(list(range(1, int(np.nanmax(lag))+1)))
    for i in range(model_param['N_NODE']):
        for j in range(model_param['N_NODE']):
            if logx:
                ax[1][0].loglog(ndata, te[i, j, :], '.-')
                ax[1][2].loglog(ndata, p[i, j, :], '.-')
                ax[0][2].semilogx(ndata, lag[i, j, :], '.-')
            else:
                ax[1][0].semilogy(ndata, te[i, j, :], '.-')
                ax[1][2].semilogy(ndata, p[i, j, :], '.-')
                ax[0][2].plot(ndata, lag[i, j, :], '.-')
    
    if fig_fname is not None:
        plt.savefig(fig_fname, dpi=300)
        
    plt.show()

## 1. Pure noise - Timestep Test

In [3]:
def DynSys_func(param):
    DS = DynSys(param)
    DS.compute()
    return np.copy(DS.data)

idtxl_settings = {
    'dim_order'       : 'ps',
    'method'          : 'MultivariateTE',
    'cmi_estimator'   : 'JidtGaussianCMI',
    'max_lag_sources' : 5,
    'min_lag_sources' : 1}

# Set parameters
model_param_noisepure = {
    'method'      : noisePure,
    'N_NODE'      : 12,             # Number of channels 
    'T_TOT'       : 10,             # seconds, Total simulation time
    'DT'          : 0.2,            # seconds, Binned optical recording resolution
    'STD'         : 1               # Standard deviation of random data
}

# Set parameters
model_param_lpfsub = {
    'method'      : noiseLPF,
    'N_NODE'      : 12,             # Number of channels 
    'T_TOT'       : 10,             # seconds, Total simulation time
    'TAU_CONV'    : 0.5,            # seconds, Ca indicator decay constant
    'DT_MICRO'    : 0.001,          # seconds, Neuronal spike timing resolution
    'DT'          : 0.2,            # seconds, Binned optical recording resolution
    'STD'         : 1               # Standard deviation of random data
}

# Set parameters
model_param_dynsys = {
    'method'  : DynSys_func,
    'ALPHA'   : 0.1,  # 1-connectivity strength
    'N_NODE'  : 12,   # Number of variables
    'N_DATA'  : 4000, # Number of timesteps
    'MAG'     : 0.0,    # Magnitude of input
    'T'       : 20,   # Period of input oscillation
    'STD'     : 0.2   # STD of neuron noise
}

model_param_all = {
    "purenoise"   : model_param_noisepure,
    "lpfsubnoise" : model_param_lpfsub,
    "dynsys"      : model_param_dynsys
}


In [ ]:
%%time

N_NODE = 12
N_STEP = 40
T_STEP = idtxl_settings['max_lag_sources'] + 1
ndata_lst = (2 * 10**(np.linspace(1.0, 2.9, N_STEP))).astype(int)

# True connectivity matrix for this problem
DS_TMP = DynSys(model_param_dynsys)
TRUE_CONN_DS = DS_TMP.M.transpose()
TRUE_CONN_DS[TRUE_CONN_DS == 0] = np.nan

TRUE_CONN_DICT = {
    "purenoise"   : np.full((N_NODE, N_NODE), np.nan),
    "lpfsubnoise" : np.full((N_NODE, N_NODE), np.nan),
    "dynsys"   :    TRUE_CONN_DS
}

#####################
# Width/Depth Analysis
#####################
idtxl_settings['dim_order'] = 'ps'
idtxl_methods = ['BivariateMI', 'MultivariateMI', 'BivariateTE', 'MultivariateTE']


for method in idtxl_methods:
    for analysis in ['width', 'depth']:
        idtxl_settings['method'] = method
        
        for modelname, model_param in model_param_all.items():
            TRUE_CONN = TRUE_CONN_DICT[modelname]

            ndata_eff = np.zeros(N_STEP, dtype=int)
            te_results = np.zeros((3, N_NODE, N_NODE, N_STEP))

            for i, ndata in enumerate(ndata_lst):
                print(analysis, method, modelname, ndata)
                
                model_param['N_NODE'] = N_NODE
                
                if analysis == 'width':
                    # Generate whole data once
                    idtxl_settings['dim_order'] = 'ps'
                    if 'DT' in model_param:
                        model_param['T_TOT']  = ndata * T_STEP * model_param['DT'] 
                    else:
                        model_param['N_DATA'] = ndata * T_STEP
                    data = model_param['method'](model_param)
                    ndata_eff[i] = data.shape[1]
                else:
                    # Generate each trial independently, then concatenate
                    idtxl_settings['dim_order'] = 'rsp'
                    if 'DT' in model_param:
                        model_param['T_TOT' ] = T_STEP * model_param['DT']
                    else:
                        model_param['N_DATA'] = T_STEP
                    
                    data = np.array([model_param['method'](model_param) for j in range(ndata)])
                    data = data.transpose((0, 2, 1))
                    ndata_eff[i] = data.shape[0]
                    
                
                # Run calculation
                rez = idtxlParallelCPU(data, idtxl_settings)

                # Parse Data
                te_results[..., i] = np.array(idtxlResultsParse(rez, N_NODE, method=method, storage='matrix'))

            # Plot
            fname = modelname + '_' + str(N_NODE) + method + '_' + analysis
            testplots(ndata_eff, te_results, TRUE_CONN, logx=True, h5_fname=fname+'.h5', fig_fname=fname+'.png')

width BivariateMI purenoise 20
Adding data with properties: 12 processes, 120 samples, 1 replications
overwriting existing data

Target: 6 - testing sources [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11]
Target: 0 - testing sources [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Target: 4 - testing sources [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11]
Target: 2 - testing sources [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Target: 3 - testing sources [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11]
Target: 5 - testing sources [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11]

Target: 1 - testing sources [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


---------------------------- (1) include source candidates

---------------------------- (1) include source candidates
---------------------------- (1) include source candidates
candidate set current source: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]




candidate set current source: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]


---------------------------- (1) include source candidates
---------------------------- (1) in

 -- not significant
 -- not significant
 -- not significant
candidate set current source: [(7, 1), (7, 2), (7, 3), (7, 4), (7, 5)]
 -- not significant
candidate set current source: [(9, 1), (9, 2), (9, 3), (9, 4), (9, 5)]
testing candidate: (7, 1) maximum statistic, n_perm: 200
candidate set current source: [(6, 1), (6, 2), (6, 3), (6, 4), (6, 5)]
candidate set current source: [(8, 1), (8, 2), (8, 3), (8, 4), (8, 5)]
 -- not significant
candidate set current source: [(6, 1), (6, 2), (6, 3), (6, 4), (6, 5)]
testing candidate: (6, 1) maximum statistic, n_perm: 200
 -- not significant
testing candidate: (6, 1) maximum statistic, n_perm: 200
testing candidate: (8, 3) maximum statistic, n_perm: 200
candidate set current source: [(7, 1), (7, 2), (7, 3), (7, 4), (7, 5)]
testing candidate: (9, 2) maximum statistic, n_perm: 200
candidate set current source: [(6, 1), (6, 2), (6, 3), (6, 4), (6, 5)]
 -- not significant
 -- not significant
 -- not significant
testing candidate: (6, 5) maximum stat

selected variables: [(8, 4)]

---------------------------- (3) final statistics
omnibus test, n_perm: 500
 -- not significant
selected variables: [(0, 3), (0, 5)]
omnibus test, n_perm: 500
selected candidates current source: [(5, 5)]
testing candidate: (1, 4) maximum statistic, n_perm: 200
 -- significant
testing candidate: (0, 5) maximum statistic, n_perm: 200

 -- significant
 -- significant
candidate set current source: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
testing candidate: (1, 2) maximum statistic, n_perm: 200
selected candidates current source: [(10, 1)]
selected vars sources [(5, 0), (9, 0)]
 -- significant
sequential maximum statistic, n_perm: 500

---------------------------- (3) final statistics
selected candidates current source: [(9, 5)]
 -- not significant
 -- not significant
 -- significant

sequential maximum statistic, n_perm: 500
candidate set current source: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
 -- not significant
candidate set current source: [(2, 1), (2, 2),


---------------------------- (2) prune candidates
selected vars sources [(0, 1)]
 -- not significant
candidate set current source: [(11, 1), (11, 2), (11, 3), (11, 4), (11, 5)]
selected candidates current source: [(0, 4)]
 -- not significant
candidate set current source: [(8, 1), (8, 2), (8, 3), (8, 4), (8, 5)]
 -- significant
 -- not significant
testing candidate: (11, 2) maximum statistic, n_perm: 200

---------------------------- (3) final statisticscandidate set current source: [(8, 1), (8, 2), (8, 3), (8, 4), (8, 5)]

testing candidate: (8, 2) maximum statistic, n_perm: 200
 -- not significant
testing candidate: (8, 3) maximum statistic, n_perm: 200
candidate set current source: [(11, 1), (11, 2), (11, 3), (11, 4), (11, 5)]
testing candidate: (11, 3) maximum statistic, n_perm: 200
selected variables: [(0, 4)]
 -- not significant
candidate set current source: [(9, 1), (9, 2), (9, 3), (9, 4), (9, 5)]
omnibus test, n_perm: 500
testing candidate: (9, 3) maximum statistic, n_perm: 200

testing candidate: (4, 5) maximum statistic, n_perm: 200
candidate set current source: [(4, 1), (4, 2), (4, 3), (4, 4), (4, 5)]
testing candidate: (5, 2) maximum statistic, n_perm: 200
testing candidate: (4, 4) maximum statistic, n_perm: 200
 -- not significant
testing candidate: (4, 1) maximum statistic, n_perm: 200
candidate set current source: [(5, 1), (5, 2), (5, 3), (5, 4), (5, 5)]
candidate set current source: [(3, 1), (3, 2), (3, 3), (3, 4), (3, 5)]
testing candidate: (3, 4) maximum statistic, n_perm: 200
 -- not significant
 -- not significant
candidate set current source: [(3, 1), (3, 2), (3, 3), (3, 4), (3, 5)]
width BivariateMI purenoise 22
Adding data with properties: 12 processes, 132 samples, 1 replications
overwriting existing data
 -- not significant
candidate set current source: [(5, 1), (5, 2), (5, 3), (5, 4), (5, 5)]
testing candidate: (5, 2) maximum statistic, n_perm: 200
testing candidate: (3, 5) maximum statistic, n_perm: 200
 -- not significant
candidate set curr

...

---------------------------- (2) prune candidates
selected vars sources [(2, 3)]
 -- not significant
 -- not significant
selected candidates current source: [(2, 2)]
candidate set current source: [(2, 1), (2, 2), (2, 3), (2, 4), (2, 5)]
candidate set current source: [(2, 1), (2, 2), (2, 3), (2, 4), (2, 5)]
 -- significant

Target: 9 - testing sources [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11]
---------------------------- (3) final statistics
 -- not significant

selected variables: [(2, 2)]

---------------------------- (1) include source candidatescandidate set current source: [(11, 1), (11, 2), (11, 3), (11, 4), (11, 5)]

testing candidate: (11, 5) maximum statistic, n_perm: 200
candidate set current source: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]
omnibus test, n_perm: 500
testing candidate: (2, 1) maximum statistic, n_perm: 200
testing candidate: (2, 5) maximum statistic, n_perm: 200
 -- not significant
 -- significant
candidate set current source: [(10, 1), (10, 2), (10, 3), (10, 4), (10

candidate set current source: [(6, 1), (6, 2), (6, 3), (6, 4), (6, 5)]
candidate set current source: [(4, 1), (4, 2), (4, 3), (4, 4), (4, 5)]
 -- not significant
candidate set current source: [(11, 1), (11, 2), (11, 3), (11, 4), (11, 5)]
 -- not significant
candidate set current source: [(6, 1), (6, 2), (6, 3), (6, 4), (6, 5)]
 -- not significant
candidate set current source: [(11, 1), (11, 2), (11, 3), (11, 4), (11, 5)]
testing candidate: (6, 2) maximum statistic, n_perm: 200
testing candidate: (4, 4) maximum statistic, n_perm: 200
testing candidate: (11, 2) maximum statistic, n_perm: 200
testing candidate: (6, 5) maximum statistic, n_perm: 200
testing candidate: (11, 5) maximum statistic, n_perm: 200
final source samples: [(3, 3), (5, 5), (5, 3), (10, 4)]
final target samples: []

 -- not significant

candidate set current source: [(5, 1), (5, 2), (5, 3), (5, 4), (5, 5)]
 -- not significant
candidate set current source: [(7, 1), (7, 2), (7, 3), (7, 4), (7, 5)]
 -- not significant

--

candidate set current source: [(2, 1), (2, 2), (2, 3), (2, 4), (2, 5)]
 -- not significant
candidate set current source: [(2, 1), (2, 2), (2, 3), (2, 4), (2, 5)]
 -- not significant
candidate set current source: [(3, 1), (3, 2), (3, 3), (3, 4), (3, 5)]
 -- not significant
 -- not significant
candidate set current source: [(2, 1), (2, 2), (2, 3), (2, 4), (2, 5)]
candidate set current source: [(3, 1), (3, 2), (3, 3), (3, 4), (3, 5)]
 -- not significant
candidate set current source: [(2, 1), (2, 2), (2, 3), (2, 4), (2, 5)]
testing candidate: (2, 1) maximum statistic, n_perm: 200
testing candidate: (3, 4) maximum statistic, n_perm: 200
testing candidate: (2, 3) maximum statistic, n_perm: 200
testing candidate: (3, 3) maximum statistic, n_perm: 200
testing candidate: (3, 1) maximum statistic, n_perm: 200
testing candidate: (2, 1) maximum statistic, n_perm: 200
testing candidate: (2, 5) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(3, 1), (3, 2), (3, 3), 

testing candidate: (11, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(11, 1), (11, 2), (11, 3), (11, 4), (11, 5)]
testing candidate: (11, 4) maximum statistic, n_perm: 200
 -- not significant
testing candidate: (11, 4) maximum statistic, n_perm: 200
candidate set current source: [(10, 1), (10, 2), (10, 3), (10, 4), (10, 5)]
 -- not significant
 -- not significant
candidate set current source: [(10, 1), (10, 2), (10, 3), (10, 4), (10, 5)]
candidate set current source: [(10, 1), (10, 2), (10, 3), (10, 4), (10, 5)]
 -- not significant

---------------------------- (2) prune candidatestesting candidate: (11, 5) maximum statistic, n_perm: 200

no sources selected, nothing to prune ...

---------------------------- (3) final statistics
 -- not significant
no sources selected ...
testing candidate: (10, 5) maximum statistic, n_perm: 200
final source samples: []

---------------------------- (2) prune candidatesfinal target samples: []

testing candidate

testing candidate: (3, 5) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(4, 1), (4, 2), (4, 3), (4, 4), (4, 5)]
testing candidate: (3, 1) maximum statistic, n_perm: 200
testing candidate: (5, 4) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(6, 1), (6, 2), (6, 3), (6, 4), (6, 5)]
 -- not significant
testing candidate: (4, 5) maximum statistic, n_perm: 200
candidate set current source: [(4, 1), (4, 2), (4, 3), (4, 4), (4, 5)]
 -- not significant
candidate set current source: [(4, 1), (4, 2), (4, 3), (4, 4), (4, 5)]
 -- not significant
candidate set current source: [(6, 1), (6, 2), (6, 3), (6, 4), (6, 5)]
testing candidate: (6, 4) maximum statistic, n_perm: 200
testing candidate: (4, 5) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(5, 1), (5, 2), (5, 3), (5, 4), (5, 5)]
testing candidate: (4, 4) maximum statistic, n_perm: 200
testing candidate: (6, 2) maximum statistic, n_perm: 20